In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
%run functions.ipynb
configurate_display()

# Introduction

Predicction: if an user is going to adopt after the trial.

Identity: user (field: user_id)

Snapshot: third trial day* (field: days_since_trial_start)

Target: user becomes customer (field: customer)  


*Goal: "ideally in the first days of trial" It will be explained later.

# Variables

In [3]:
snapshot = 3
period_init = pd.to_datetime('20201201', format='%Y%m%d')
period_end = pd.to_datetime('20201231', format='%Y%m%d')

# Data reading

In [4]:
df_onboarding = read_file_tsv("tables/users_onboarding_paywall.tsv")
df_onboarding.shape

(42239, 20)

In [5]:
df_activities = read_file_tsv("tables/activities_per_day.tsv")
df_activities.shape

(109764, 23)

# Data cleaning

In [6]:
#Unique users on onboarding. 
'''There are repeated users in onboarding table. I couldn't find any pattern in a reduced time so I decided to ignore them 
because there are less than 500 cases.'''

df_onboarding.drop_duplicates(subset='user_id', keep = False, inplace = True)

In [7]:
#*Prediction "ideally in the first days of trial"
'''
Because the goal say: ideally in the first days of trial, I decided to train with snapshots close to the first trial day.

Analyzing the data, exists activities which days_since_trial_start field is unknown so that samples will be dropped 
because I can find a commun sense or logical reason. Ideally should be understand the reasons and use that data if we are 
sure is not future information.

I will analice which trial day has the best trade-off between min trial day and max user activity (more samples).

The following table shows that if I choose activities where days_since_trial_start are in [0, 1, 2] I can use 91% of the samples. 
I think that is a good final quantity for this demo.
Ideally, this decision may consider the usage of the model and the stakeholders opinion.
'''

first_activity = df_activities.groupby(['user_id'])['days_since_trial_start'].min().to_frame().reset_index()
first_activity.groupby('days_since_trial_start').count().reset_index()

,days_since_trial_start,user_id
0,0,22279
1,1,823
2,2,679
3,3,610
4,4,599
5,5,569
6,6,573


In [8]:
df_activities = df_activities[df_activities.days_since_trial_start.isin({0,1,2})]

In [9]:
#Samples without enough maduration: they need at least 3 days of activity after the start of the trial.
df_onboarding = df_onboarding[str_to_dt(df_onboarding, 'subscription_at') <= (period_end - np.timedelta64(snapshot,'D'))]

In [10]:
#Parse data
df_onboarding['onboarding_home_at_dt'] = str_to_dt(df_onboarding, 'onboarding_home_at')
df_onboarding['subscription_at_dt'] = str_to_dt(df_onboarding, 'subscription_at')

# Feature building

### Onboarding features

I assume that data of the onboarding table is obtained before the trial starts. Except: customer

In [11]:
#Subscription delay hypothesis: if an user think more on the subscription that could be related with no adoption.
df_onboarding['min_btw_onboarding_home_and_subscripition'] = df_onboarding['subscription_at_dt'] - df_onboarding['onboarding_home_at_dt']
to_minutes(df_onboarding, 'min_btw_onboarding_home_and_subscripition')

In [12]:
#Seasonality hypothesis: if the trial is during the salary period the users could afford the subscription.
df_onboarding['subscription_seasonality'] = df_onboarding['subscription_at_dt'].dt.day

In [13]:
#Continuous version hypothesis: the higher version number, the later mobile model.
split_version = df_onboarding['os_version'].str.split('.')
df_onboarding['os_version_cont'] = split_version.str[:2].str.join('.')
df_onboarding['os_version_cont'] = pd.to_numeric(df_onboarding.os_version_cont, errors='coerce')

In [14]:
#Simplified timezone hypothesis: too many branches of the data could make a categorical feature unusable.
split_timezone = df_onboarding['timezone'].str.split('/')
df_onboarding['timezone_continent'] = split_timezone.str[0]

In [15]:
df_onboarding_features = df_onboarding[['user_id', 'os_name', 'os_version_cont', 'country', 'timezone_continent', 'locale', 
                                        'source', 'level', 'age', 'signup_provider', 'price', 'currency', 
                                        'payment_platform', 'min_btw_onboarding_home_and_subscripition', 
                                        'subscription_seasonality']]

### Activities features

In [16]:
#I will consider the data per day because disconnections are not necessary for the quantification of the activities.
df_activities_group_per_day = df_activities.groupby(['user_id', 'days_since_trial_start']).agg(
    n_activities_game = pd.NamedAgg(column = "n_activities_game", aggfunc= "sum"),
    n_activities_video = pd.NamedAgg(column = "n_activities_video", aggfunc= "sum"),
    n_activities_traceable = pd.NamedAgg(column = "n_activities_traceable", aggfunc= "sum"),
    n_activities_audiobook = pd.NamedAgg(column = "n_activities_audiobook", aggfunc= "sum"),
    n_activities_completed_game = pd.NamedAgg(column = "n_activities_completed_game", aggfunc= "sum"),
    n_activities_completed_video = pd.NamedAgg(column = "n_activities_completed_video", aggfunc= "sum"),
    n_activities_completed_traceable = pd.NamedAgg(column = "n_activities_completed_traceable", aggfunc= "sum"),
    n_activities_completed_audiobook = pd.NamedAgg(column = "n_activities_completed_audiobook", aggfunc= "sum"),

    duration_game = pd.NamedAgg(column = "total_duration_game", aggfunc= "sum"),
    duration_video = pd.NamedAgg(column = "total_duration_video", aggfunc= "sum"),
    duration_traceable = pd.NamedAgg(column = "total_duration_traceable", aggfunc= "sum"),
    duration_audiobook = pd.NamedAgg(column = "total_duration_audiobook", aggfunc= "sum"),
    
    qty_sessions_per_day = pd.NamedAgg(column = "n_sessions", aggfunc= "count"),
    
    
).reset_index()

#Total per day
df_activities_group_per_day['n_activities_tot'] = df_activities_group_per_day['n_activities_game'] + df_activities_group_per_day['n_activities_video'] + df_activities_group_per_day['n_activities_traceable'] + df_activities_group_per_day['n_activities_audiobook']
df_activities_group_per_day['n_activities_completed_tot'] = df_activities_group_per_day['n_activities_completed_game'] + df_activities_group_per_day['n_activities_completed_video'] + df_activities_group_per_day['n_activities_completed_traceable'] + df_activities_group_per_day['n_activities_completed_audiobook']
df_activities_group_per_day['duration_tot'] = df_activities_group_per_day['duration_game'] + df_activities_group_per_day['duration_video'] + df_activities_group_per_day['duration_traceable'] + df_activities_group_per_day['duration_audiobook']


In [17]:
df_activities_features = df_activities_group_per_day.groupby('user_id').agg(
    #Total
    n_activities_game_tot = pd.NamedAgg(column = "n_activities_game", aggfunc= "sum"),
    n_activities_video_tot = pd.NamedAgg(column = "n_activities_video", aggfunc= "sum"),
    n_activities_traceable_tot = pd.NamedAgg(column = "n_activities_traceable", aggfunc= "sum"),
    n_activities_audiobook_tot = pd.NamedAgg(column = "n_activities_audiobook", aggfunc= "sum"),
    n_activities_completed_game_tot = pd.NamedAgg(column = "n_activities_completed_game", aggfunc= "sum"),
    n_activities_completed_video_tot = pd.NamedAgg(column = "n_activities_completed_video", aggfunc= "sum"),
    n_activities_completed_traceable_tot = pd.NamedAgg(column = "n_activities_completed_traceable", aggfunc= "sum"),
    n_activities_completed_audiobook_tot = pd.NamedAgg(column = "n_activities_completed_audiobook", aggfunc= "sum"),
    
    duration_game_tot = pd.NamedAgg(column = "duration_game", aggfunc= "sum"),
    duration_video_tot = pd.NamedAgg(column = "duration_video", aggfunc= "sum"),
    duration_traceable_tot = pd.NamedAgg(column = "duration_traceable", aggfunc= "sum"),
    duration_audiobook_tot = pd.NamedAgg(column = "duration_audiobook", aggfunc= "sum"),
    
    qty_sessions_per_day_tot = pd.NamedAgg(column = "qty_sessions_per_day", aggfunc= "sum"),
    
    #MIN
    n_activities_game_min = pd.NamedAgg(column = "n_activities_game", aggfunc= "min"),
    n_activities_video_min = pd.NamedAgg(column = "n_activities_video", aggfunc= "min"),
    n_activities_traceable_min = pd.NamedAgg(column = "n_activities_traceable", aggfunc= "min"),
    n_activities_audiobook_min = pd.NamedAgg(column = "n_activities_audiobook", aggfunc= "min"),
    n_activities_completed_game_min = pd.NamedAgg(column = "n_activities_completed_game", aggfunc= "min"),
    n_activities_completed_video_min = pd.NamedAgg(column = "n_activities_completed_video", aggfunc= "min"),
    n_activities_completed_traceable_min = pd.NamedAgg(column = "n_activities_completed_traceable", aggfunc= "min"),
    n_activities_completed_audiobook_min = pd.NamedAgg(column = "n_activities_completed_audiobook", aggfunc= "min"),
    
    duration_game_min = pd.NamedAgg(column = "duration_game", aggfunc= "min"),
    duration_video_min = pd.NamedAgg(column = "duration_video", aggfunc= "min"),
    duration_traceable_min = pd.NamedAgg(column = "duration_traceable", aggfunc= "min"),
    duration_audiobook_min = pd.NamedAgg(column = "duration_audiobook", aggfunc= "min"),
    
    qty_sessions_per_day_min = pd.NamedAgg(column = "qty_sessions_per_day", aggfunc= "min"),
    
    #MAX
    n_activities_game_max = pd.NamedAgg(column = "n_activities_game", aggfunc= "max"),
    n_activities_video_max = pd.NamedAgg(column = "n_activities_video", aggfunc= "max"),
    n_activities_traceable_max = pd.NamedAgg(column = "n_activities_traceable", aggfunc= "max"),
    n_activities_audiobook_max = pd.NamedAgg(column = "n_activities_audiobook", aggfunc= "max"),
    n_activities_completed_game_max = pd.NamedAgg(column = "n_activities_completed_game", aggfunc= "max"),
    n_activities_completed_video_max = pd.NamedAgg(column = "n_activities_completed_video", aggfunc= "max"),
    n_activities_completed_traceable_max = pd.NamedAgg(column = "n_activities_completed_traceable", aggfunc= "max"),
    n_activities_completed_audiobook_max = pd.NamedAgg(column = "n_activities_completed_audiobook", aggfunc= "max"),
    
    duration_game_max = pd.NamedAgg(column = "duration_game", aggfunc= "max"),
    duration_video_max = pd.NamedAgg(column = "duration_video", aggfunc= "max"),
    duration_traceable_max = pd.NamedAgg(column = "duration_traceable", aggfunc= "max"),
    duration_audiobook_max = pd.NamedAgg(column = "duration_audiobook", aggfunc= "max"),
    
    qty_sessions_per_day_max = pd.NamedAgg(column = "qty_sessions_per_day", aggfunc= "max"),
    
    #AVG
    n_activities_game_avg = pd.NamedAgg(column = "n_activities_game", aggfunc= np.mean),
    n_activities_video_avg = pd.NamedAgg(column = "n_activities_video", aggfunc= np.mean),
    n_activities_traceable_avg = pd.NamedAgg(column = "n_activities_traceable", aggfunc= np.mean),
    n_activities_audiobook_avg = pd.NamedAgg(column = "n_activities_audiobook", aggfunc= np.mean),
    n_activities_completed_game_avg = pd.NamedAgg(column = "n_activities_completed_game", aggfunc= np.mean),
    n_activities_completed_video_avg = pd.NamedAgg(column = "n_activities_completed_video", aggfunc= np.mean),
    n_activities_completed_traceable_avg = pd.NamedAgg(column = "n_activities_completed_traceable", aggfunc= np.mean),
    n_activities_completed_audiobook_avg = pd.NamedAgg(column = "n_activities_completed_audiobook", aggfunc= np.mean),
    
    duration_game_avg = pd.NamedAgg(column = "duration_game", aggfunc= np.mean),
    duration_video_avg = pd.NamedAgg(column = "duration_video", aggfunc= np.mean),
    duration_traceable_avg = pd.NamedAgg(column = "duration_traceable", aggfunc= np.mean),
    duration_audiobook_avg = pd.NamedAgg(column = "duration_audiobook", aggfunc= np.mean),
    
    qty_sessions_per_day_avg = pd.NamedAgg(column = "qty_sessions_per_day", aggfunc= np.mean),
    
    #Qty days
    connected_day_min = pd.NamedAgg(column = "days_since_trial_start", aggfunc= "min"),
    connected_day_max = pd.NamedAgg(column = "days_since_trial_start", aggfunc= "max"),
    connected_days_qty = pd.NamedAgg(column = "days_since_trial_start", aggfunc= "sum"),

    
).reset_index()

In [18]:
#ACTIVITIES PORCENTAJES
#Total
df_activities_features['n_activities_tot'] = df_activities_features['n_activities_game_tot'] + df_activities_features['n_activities_video_tot'] + df_activities_features['n_activities_traceable_tot'] + df_activities_features['n_activities_audiobook_tot']
df_activities_features['n_activities_completed_tot'] = df_activities_features['n_activities_completed_game_tot'] + df_activities_features['n_activities_completed_video_tot'] + df_activities_features['n_activities_completed_traceable_tot'] + df_activities_features['n_activities_completed_audiobook_tot']

#Percentage of complete activities
df_activities_features['n_activities_completed_porc'] = df_activities_features['n_activities_completed_tot']/df_activities_features['n_activities_tot']

df_activities_features['n_activities_completed_game_porc'] = df_activities_features['n_activities_completed_game_tot']/ df_activities_features['n_activities_game_tot']
df_activities_features['n_activities_completed_video_porc'] = df_activities_features['n_activities_completed_video_tot']/ df_activities_features['n_activities_video_tot']
df_activities_features['n_activities_completed_traceable_porc'] = df_activities_features['n_activities_completed_traceable_tot']/ df_activities_features['n_activities_traceable_tot']
df_activities_features['n_activities_completed_audiobook_porc'] = df_activities_features['n_activities_completed_audiobook_tot']/ df_activities_features['n_activities_audiobook_tot']

#Share type of activity
df_activities_features['n_activities_completed_game_share'] = df_activities_features['n_activities_completed_game_tot']/ df_activities_features['n_activities_completed_tot']
df_activities_features['n_activities_completed_video_share'] = df_activities_features['n_activities_completed_video_tot']/ df_activities_features['n_activities_completed_tot']
df_activities_features['n_activities_completed_traceable_share'] = df_activities_features['n_activities_completed_traceable_tot']/ df_activities_features['n_activities_completed_tot']
df_activities_features['n_activities_completed_audiobook_share'] = df_activities_features['n_activities_completed_audiobook_tot']/ df_activities_features['n_activities_completed_tot']

df_activities_features['n_activities_game_share'] = df_activities_features['n_activities_game_tot']/ df_activities_features['n_activities_tot']
df_activities_features['n_activities_video_share'] = df_activities_features['n_activities_video_tot']/ df_activities_features['n_activities_tot']
df_activities_features['n_activities_traceable_share'] = df_activities_features['n_activities_traceable_tot']/ df_activities_features['n_activities_tot']
df_activities_features['n_activities_audiobook_share'] = df_activities_features['n_activities_audiobook_tot']/ df_activities_features['n_activities_tot']

In [19]:
#DURATION PERCENTAGES

#Total
df_activities_features['duration_tot'] = df_activities_features['duration_game_tot'] + df_activities_features['duration_video_tot'] + df_activities_features['duration_traceable_tot'] + df_activities_features['duration_audiobook_tot']

#Share type of activity
df_activities_features['duration_game_share'] = df_activities_features['duration_game_tot']/ df_activities_features['duration_tot']
df_activities_features['duration_video_share'] = df_activities_features['duration_video_tot']/ df_activities_features['duration_tot']
df_activities_features['duration_traceable_share'] = df_activities_features['duration_traceable_tot']/ df_activities_features['duration_tot']
df_activities_features['duration_audiobook_share'] = df_activities_features['duration_audiobook_tot']/ df_activities_features['duration_tot']

In [20]:
#TREND hypothesis: if the activity increase that implicate more engage
df_activities_features = pd.merge(df_activities_features, df_activities_group_per_day[['user_id', 'days_since_trial_start', 'n_activities_tot', 'n_activities_completed_tot', 'duration_tot']], how='left', left_on=['user_id', 'connected_day_min'], right_on=['user_id', 'days_since_trial_start'], suffixes = ('','_first_day'))
df_activities_features = pd.merge(df_activities_features, df_activities_group_per_day[['user_id', 'days_since_trial_start', 'n_activities_tot', 'n_activities_completed_tot', 'duration_tot']], how='left', left_on=['user_id', 'connected_day_max'], right_on=['user_id', 'days_since_trial_start'], suffixes = ('','_last_day'))
df_activities_features.rename(columns = {'days_since_trial_start':'days_since_trial_start_first_day'}, inplace = True)


In [21]:
df_activities_features.loc[(df_activities_features['n_activities_tot_first_day'] == 0) & (df_activities_features['n_activities_tot_last_day'] != 0), 'n_activities_trend'] = 999
df_activities_features.loc[(df_activities_features['n_activities_tot_first_day'] == 0) & (df_activities_features['n_activities_tot_last_day'] == 0), 'n_activities_trend'] = 1
df_activities_features.loc[df_activities_features['n_activities_tot_first_day'] != 0, 'n_activities_trend'] = df_activities_features['n_activities_tot_last_day']/df_activities_features['n_activities_tot_first_day']

df_activities_features.loc[(df_activities_features['n_activities_completed_tot_first_day'] == 0) & (df_activities_features['n_activities_completed_tot_last_day'] != 0), 'n_activities_completed_trend'] = 999
df_activities_features.loc[(df_activities_features['n_activities_completed_tot_first_day'] == 0) & (df_activities_features['n_activities_completed_tot_last_day'] == 0), 'n_activities_completed_trend'] = 1
df_activities_features.loc[df_activities_features['n_activities_completed_tot_first_day'] != 0, 'n_activities_completed_trend'] = df_activities_features['n_activities_completed_tot_last_day']/df_activities_features['n_activities_completed_tot_first_day']

df_activities_features.loc[(df_activities_features['duration_tot_first_day'] == 0) & (df_activities_features['duration_tot_last_day'] != 0), 'duration_trend'] = 999
df_activities_features.loc[(df_activities_features['duration_tot_first_day'] == 0) & (df_activities_features['duration_tot_last_day'] == 0), 'duration_trend'] = 1
df_activities_features.loc[df_activities_features['duration_tot_first_day'] != 0, 'duration_trend'] = df_activities_features['duration_tot_last_day']/df_activities_features['duration_tot_first_day']


# Features union

In [22]:
df_features = pd.merge(df_onboarding_features, df_activities_features, how='left', left_on=['user_id'], right_on=['user_id'])

# Target

In [23]:
df = pd.merge(df_features, df_onboarding[['user_id', 'customer']], how='left', left_on=['user_id'], right_on=['user_id'])
df.rename(columns = {'customer': 'target'}, inplace = True)
df.shape

(36025, 102)

# Fill Nan

I fill nan with -999 to easily identify Nan cases.
It could be improved by analysing each feature and assigning the best replace for each.

In [24]:
#Fill numerical features
features_float_values = {'age' : -999, 'price' : -999, 'os_version_cont' : -999, 'min_btw_onboarding_home_and_subscripition' : -999, 'n_activities_game_tot' : -999, 'n_activities_video_tot' : -999, 'n_activities_traceable_tot' : -999, 'n_activities_audiobook_tot' : -999, 'n_activities_completed_game_tot' : -999, 'n_activities_completed_video_tot' : -999, 'n_activities_completed_traceable_tot' : -999, 'n_activities_completed_audiobook_tot' : -999, 'duration_game_tot' : -999, 'duration_video_tot' : -999, 'duration_traceable_tot' : -999, 'duration_audiobook_tot' : -999, 'qty_sessions_per_day_tot' : -999, 'n_activities_game_min' : -999, 'n_activities_video_min' : -999, 'n_activities_traceable_min' : -999, 'n_activities_audiobook_min' : -999, 'n_activities_completed_game_min' : -999, 'n_activities_completed_video_min' : -999, 'n_activities_completed_traceable_min' : -999, 'n_activities_completed_audiobook_min' : -999, 'duration_game_min' : -999, 'duration_video_min' : -999, 'duration_traceable_min' : -999, 'duration_audiobook_min' : -999, 'qty_sessions_per_day_min' : -999, 'n_activities_game_max' : -999, 'n_activities_video_max' : -999, 'n_activities_traceable_max' : -999, 'n_activities_audiobook_max' : -999, 'n_activities_completed_game_max' : -999, 'n_activities_completed_video_max' : -999, 'n_activities_completed_traceable_max' : -999, 'n_activities_completed_audiobook_max' : -999, 'duration_game_max' : -999, 'duration_video_max' : -999, 'duration_traceable_max' : -999, 'duration_audiobook_max' : -999, 'qty_sessions_per_day_max' : -999, 'n_activities_game_avg' : -999, 'n_activities_video_avg' : -999, 'n_activities_traceable_avg' : -999, 'n_activities_audiobook_avg' : -999, 'n_activities_completed_game_avg' : -999, 'n_activities_completed_video_avg' : -999, 'n_activities_completed_traceable_avg' : -999, 'n_activities_completed_audiobook_avg' : -999, 'duration_game_avg' : -999, 'duration_video_avg' : -999, 'duration_traceable_avg' : -999, 'duration_audiobook_avg' : -999, 'qty_sessions_per_day_avg' : -999, 'connected_day_min' : -999, 'connected_day_max' : -999, 'connected_days_qty' : -999, 'n_activities_tot' : -999, 'n_activities_completed_tot' : -999, 'n_activities_completed_porc' : -999, 'n_activities_completed_game_porc' : -999, 'n_activities_completed_video_porc' : -999, 'n_activities_completed_traceable_porc' : -999, 'n_activities_completed_audiobook_porc' : -999, 'n_activities_completed_game_share' : -999, 'n_activities_completed_video_share' : -999, 'n_activities_completed_traceable_share' : -999, 'n_activities_completed_audiobook_share' : -999, 'n_activities_game_share' : -999, 'n_activities_video_share' : -999, 'n_activities_traceable_share' : -999, 'n_activities_audiobook_share' : -999, 'duration_tot' : -999, 'duration_game_share' : -999, 'duration_video_share' : -999, 'duration_traceable_share' : -999, 'duration_audiobook_share' : -999, 'days_since_trial_start_first_day' : -999, 'n_activities_tot_first_day' : -999, 'n_activities_completed_tot_first_day' : -999, 'duration_tot_first_day' : -999, 'days_since_trial_start_last_day' : -999, 'n_activities_tot_last_day' : -999, 'n_activities_completed_tot_last_day' : -999, 'duration_tot_last_day' : -999, 'n_activities_trend' : -999, 'n_activities_completed_trend' : -999, 'duration_trend' : -999, 'subscription_seasonality' : -999}
df = df.fillna(value = features_float_values)

In [25]:
#Fill categorical features
features_categorical_values = {'os_name' : 'no_value', 'country' : 'no_value', 'timezone_continent' : 'no_value', 'locale' : 'no_value', 'source' : 'no_value', 'level' : 'no_value', 'signup_provider' : 'no_value', 'currency' : 'no_value', 'payment_platform' : 'no_value'}
df = df.fillna(value = features_categorical_values)

In [26]:
#Convert categorical features to dummies
categorical_feature = get_categorical_features()
df[categorical_feature] = df[categorical_feature].astype('category')
df_dummy = pd.get_dummies(df[['user_id'] + categorical_feature], columns=categorical_feature)
df = pd.merge(df, df_dummy, how='left', left_on=['user_id'], right_on=['user_id'])

In [27]:
df.head()

,user_id,os_name,os_version_cont,country,timezone_continent,locale,source,level,age,signup_provider,price,currency,payment_platform,min_btw_onboarding_home_and_subscripition,subscription_seasonality,n_activities_game_tot,n_activities_video_tot,n_activities_traceable_tot,n_activities_audiobook_tot,n_activities_completed_game_tot,n_activities_completed_video_tot,n_activities_completed_traceable_tot,n_activities_completed_audiobook_tot,duration_game_tot,duration_video_tot,duration_traceable_tot,duration_audiobook_tot,qty_sessions_per_day_tot,n_activities_game_min,n_activities_video_min,n_activities_traceable_min,n_activities_audiobook_min,n_activities_completed_game_min,n_activities_completed_video_min,n_activities_completed_traceable_min,n_activities_completed_audiobook_min,duration_game_min,duration_video_min,duration_traceable_min,duration_audiobook_min,qty_sessions_per_day_min,n_activities_game_max,n_activities_video_max,n_activities_traceable_max,n_activities_audiobook_max,n_activities_completed_game_max,n_activities_completed_video_max,n_activities_completed_traceable_max,n_activities_completed_audiobook_max,duration_game_max,duration_video_max,duration_traceable_max,duration_audiobook_max,qty_sessions_per_day_max,n_activities_game_avg,n_activities_video_avg,n_activities_traceable_avg,n_activities_audiobook_avg,n_activities_completed_game_avg,n_activities_completed_video_avg,n_activities_completed_traceable_avg,n_activities_completed_audiobook_avg,duration_game_avg,duration_video_avg,duration_traceable_avg,duration_audiobook_avg,qty_sessions_per_day_avg,connected_day_min,connected_day_max,connected_days_qty,n_activities_tot,n_activities_completed_tot,n_activities_completed_porc,n_activities_completed_game_porc,n_activities_completed_video_porc,n_activities_completed_traceable_porc,n_activities_completed_audiobook_porc,n_activities_completed_game_share,n_activities_completed_video_share,n_activities_completed_traceable_share,n_activities_completed_audiobook_share,n_activities_game_share,n_activities_video_share,n_activities_traceable_share,n_activities_audiobook_share,duration_tot,duration_game_share,duration_video_share,duration_traceable_share,duration_audiobook_share,days_since_trial_start_first_day,n_activities_tot_first_day,n_activities_completed_tot_first_day,duration_tot_first_day,days_since_trial_start_last_day,n_activities_tot_last_day,n_activities_completed_tot_last_day,duration_tot_last_day,n_activities_trend,n_activities_completed_trend,duration_trend,target,os_name_Android,os_name_iOS,os_name_iPhone OS,os_name_no_value,country_ad,country_ae,country_af,country_ag,country_ai,country_al,country_am,country_ao,country_ar,country_at,country_au,country_aw,country_az,country_ba,country_bb,country_bd,country_be,country_bg,country_bh,country_bj,country_bm,country_bn,country_bo,country_br,country_bs,country_bt,country_bw,country_by,country_bz,country_ca,country_cg,country_ch,country_ci,country_cl,country_cm,country_cn,country_co,country_cr,country_cv,country_cw,country_cy,country_cz,country_de,country_dk,country_dm,country_do,country_dz,country_ec,country_ee,country_eg,country_es,country_et,country_fi,country_fo,country_fr,country_ga,country_gb,country_gd,country_ge,country_gf,country_gh,country_gi,country_gl,country_gm,country_gp,country_gr,country_gt,country_gu,country_gy,country_hk,country_hn,country_hr,country_ht,country_hu,country_id,country_ie,country_il,country_in,country_iq,country_ir,country_is,country_it,country_je,country_jm,country_jo,country_jp,country_ke,country_kg,country_kh,country_kn,country_kr,country_kw,country_ky,country_kz,country_la,country_lb,country_lc,country_li,country_lk,country_lr,country_ls,country_lt,country_lu,country_lv,country_ly,country_ma,country_mc,country_md,country_me,country_mg,country_mk,country_ml,country_mm,country_mn,country_mo,country_mp,country_mq,country_mr,country_mt,country_mu,country_mv,country_mw,country_mx,country_my,country_mz,country_na,country_nc,country_ng,country_ni,c

In [28]:
df.shape

(36025, 437)

# Save dataset

In [29]:
write_file_csv(df, 'dataset')